In [471]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

In [475]:
#Setting up data

data = pd.read_excel('rbabondyields.xlsx') # import data from excel 
df = data.drop([0,1,2,3,4,5,6,7,8,9]) # drop non-numeric rows at the top of the sheet
df1 = df.reset_index(drop=True) # change indexing
df1.columns = ['Dates', 'ACGB 2yr', 'ACGB 3yr', 'ACGB 5yr', 'ACGB 10yr'] #rewriting columns
df1['ACGB 2yr'] = pd.to_numeric(df1['ACGB 2yr'], errors='coerce') #ensure type for data is set to numeric/float64
df1['ACGB 3yr'] = pd.to_numeric(df1['ACGB 3yr'], errors='coerce')
df1['ACGB 5yr'] = pd.to_numeric(df1['ACGB 5yr'], errors='coerce')
df1['ACGB 10yr'] = pd.to_numeric(df1['ACGB 10yr'], errors='coerce')
df1['Dates'] = pd.to_datetime(df1['Dates']) #ensure type for data is set to datetime
df1['Dates'] = pd.to_datetime(df1['Dates']).dt.date # drops time next to the date 

df2 = df1.dropna() #ignore rows that have missing data 
df3 = df2.copy()
df3

,Dates,ACGB 2yr,ACGB 3yr,ACGB 5yr,ACGB 10yr
0,2013-09-02,2.594,2.800,3.258,3.989
1,2013-09-03,2.652,2.857,3.312,4.030
2,2013-09-04,2.714,2.925,3.373,4.055
3,2013-09-05,2.752,2.968,3.412,4.106
4,2013-09-06,2.790,3.007,3.455,4.177
...,...,...,...,...,...
2656,2024-02-29,3.754,3.702,3.774,4.141
2657,2024-03-01,3.764,3.712,3.782,4.146
2658,2024-03-04,3.732,3.682,3.752,4.106
2659,2024-03-05,3.732,3.682,3.749,4.096


In [479]:
#DISCOUNT FACTORS T <= 2 

# Define the function to solve for discount factors using root_scalar
def bond_price_equation(x, coupon, FV=100, price=100):
    return (coupon / 2) * x**(1/4) + (coupon / 2) * x**(1/2) + (coupon / 2) * x**(3/4) + (FV + coupon / 2) * x - price # solving for B(0,2) as 'x' with the intermediate DFs written as a fn of x 

# Define a function to calculate the discount factor for B(0,2)
def calculate_discount_factor(coupon):
    from scipy.optimize import root_scalar
    result = root_scalar(bond_price_equation, args=(coupon,), bracket=[0.0001, 10]) # wider interval to ensure we find a root 
    return result.root

# create new discount factors to store 
df_discount_factors = pd.DataFrame()
df_discount_factors['Dates'] = df3['Dates']

# Apply the function to the 2yr ACGB column to get the discount factor for B(0,2)
df_discount_factors['2yr DF'] = df3['ACGB 2yr'].apply(calculate_discount_factor)

#then work out B(0,0.5), B(0,1), B(1.5) and store to discount factors DF. # use B(0,0) = 1 to simplify from: lnB(0.5) = lnB(0,0) + (0.5-0)/(2-0)*(lnB(0,2)-lnB(0,0)) , etc 

df_discount_factors['0.5y DF'] = df_discount_factors['2yr DF']**(1/4)
df_discount_factors['1y DF'] = df_discount_factors['2yr DF']**(1/2)
df_discount_factors['1.5y DF'] = df_discount_factors['2yr DF']**(3/4)

df_discount_factors = df_discount_factors[['Dates','0.5y DF', '1y DF', '1.5y DF', '2yr DF']]
df_discount_factors

,Dates,0.5y DF,1y DF,1.5y DF,2yr DF
0,2013-09-02,0.987196,0.974556,0.962078,0.949760
1,2013-09-03,0.986914,0.973998,0.961252,0.948673
2,2013-09-04,0.986612,0.973403,0.960370,0.947513
3,2013-09-05,0.986427,0.973038,0.959831,0.946802
4,2013-09-06,0.986242,0.972673,0.959291,0.946093
...,...,...,...,...,...
2656,2024-02-29,0.981576,0.963491,0.945740,0.928315
2657,2024-03-01,0.981528,0.963397,0.945600,0.928133
2658,2024-03-04,0.981682,0.963699,0.946046,0.928716
2659,2024-03-05,0.981682,0.963699,0.946046,0.928716


In [481]:
#DISCOUNT FACTORS 2 < T <=3 

df_merged = pd.merge(df_discount_factors, df3[['Dates', 'ACGB 3yr']], on='Dates', how='left') #add coupon column to DF data frame 

# Define the bond price equation with the previously calculated discount factors and set up to solve for for B(0,3) 
def bond_price_equation(x, coupon, df_05y, df_1y, df_15y, df_2y, FV=100, price=100):
    return (
        (coupon / 2) * df_05y + 
        (coupon / 2) * df_1y + 
        (coupon / 2) * df_15y + 
        (coupon / 2) * df_2y + 
        (coupon / 2) * (df_2y*x)**(1/2) + 
        (FV + coupon / 2) * x - price
    )
# Define a function to calculate x

def calculate_x(coupon, df_05y, df_1y, df_15y, df_2y): 
    from scipy.optimize import root_scalar
    result = root_scalar(bond_price_equation, args=(coupon, df_05y, df_1y, df_15y, df_2y), bracket=[0,1]) # including the previously calculated arguments to 'bootstrap'
    return result.root
    
# Apply the function on the merged DataFrame for B(0,3)
df_merged['3y DF'] = df_merged.apply(
    lambda row: calculate_x(row['ACGB 3yr'], row['0.5y DF'], row['1y DF'], row['1.5y DF'], row['2yr DF']),
    axis=1

)

df_merged['2.5y DF'] = (df_merged['2yr DF']*df_merged['3y DF'])**(1/2) # interpolate for 2.5y DF
df_merged = df_merged[['Dates','0.5y DF', '1y DF', '1.5y DF', '2yr DF', '2.5y DF', '3y DF']] #order the data frame in ascending order
df_merged

,Dates,0.5y DF,1y DF,1.5y DF,2yr DF,2.5y DF,3y DF
0,2013-09-02,0.987196,0.974556,0.962078,0.949760,0.934663,0.919807
1,2013-09-03,0.986914,0.973998,0.961252,0.948673,0.933340,0.918255
2,2013-09-04,0.986612,0.973403,0.960370,0.947513,0.931827,0.916402
3,2013-09-05,0.986427,0.973038,0.959831,0.946802,0.930883,0.915231
4,2013-09-06,0.986242,0.972673,0.959291,0.946093,0.929996,0.914173
...,...,...,...,...,...,...,...
2544,2024-02-29,0.981576,0.963491,0.945740,0.928315,0.911936,0.895846
2545,2024-03-01,0.981528,0.963397,0.945600,0.928133,0.911712,0.895582
2546,2024-03-04,0.981682,0.963699,0.946046,0.928716,0.912401,0.896372
2547,2024-03-05,0.981682,0.963699,0.946046,0.928716,0.912401,0.896372


In [483]:
#DISCOUNT FACTORS 3 < T <=5 

df4 = pd.merge(df_merged, df3[['Dates', 'ACGB 5yr']], on='Dates', how='left') #add 5yr coupon column to DF data frame  
df4

# Define the bond price equation with the previously calculated discount factors and set up to solve for for B(0,5) 
def bond_price_equation(x, coupon, df_05y, df_1y, df_15y, df_2y, df_25y, df_3y, FV=100, price=100):
    return (
        (coupon / 2) * df_05y + 
        (coupon / 2) * df_1y + 
        (coupon / 2) * df_15y + 
        (coupon / 2) * df_2y + 
        (coupon / 2) * df_25y + 
        (coupon / 2) * df_3y + 
        (coupon / 2) * (df_3y)**(3/4)*x**(1/4) + 
        (coupon / 2) * (df_3y*x)**(1/2) + 
        (coupon / 2) * (df_3y)**(1/4)*x**(3/4) + 
        (FV + coupon / 2) * x - price
    )
# Define a function to calculate x

def calculate_x(coupon, df_05y, df_1y, df_15y, df_2y, df_25y, df_3y):
    from scipy.optimize import root_scalar
    result = root_scalar(bond_price_equation, args=(coupon, df_05y, df_1y, df_15y, df_2y, df_25y, df_3y), bracket=[0,1])
    return result.root
# Apply the function on the merged DataFrame for B(0,5)
df4['5y DF'] = df4.apply(
    lambda row: calculate_x(row['ACGB 5yr'], row['0.5y DF'], row['1y DF'], row['1.5y DF'], row['2yr DF'], row['2.5y DF'], row['3y DF']),
    axis=1

)

#then work out B(0,3.5), B(0,4), B(4.5) and store to discount factors df4
df4['3.5y DF'] = df4['3y DF']**(3/4)*df4['5y DF']**(1/4)
df4['4y DF'] = (df4['3y DF']*df4['5y DF'])**(1/2)
df4['4.5y DF'] = df4['3y DF']**(1/4)*df4['5y DF']**(3/4)

df4 = df4[['Dates','0.5y DF', '1y DF', '1.5y DF', '2yr DF', '2.5y DF', '3y DF', '3.5y DF', '4y DF', '4.5y DF', '5y DF']]
df4

,Dates,0.5y DF,1y DF,1.5y DF,2yr DF,2.5y DF,3y DF,3.5y DF,4y DF,4.5y DF,5y DF
0,2013-09-02,0.987196,0.974556,0.962078,0.949760,0.934663,0.919807,0.901740,0.884028,0.866664,0.849641
1,2013-09-03,0.986914,0.973998,0.961252,0.948673,0.933340,0.918255,0.899998,0.882103,0.864565,0.847375
2,2013-09-04,0.986612,0.973403,0.960370,0.947513,0.931827,0.916402,0.897958,0.879886,0.862178,0.844826
3,2013-09-05,0.986427,0.973038,0.959831,0.946802,0.930883,0.915231,0.896665,0.878476,0.860656,0.843198
4,2013-09-06,0.986242,0.972673,0.959291,0.946093,0.929996,0.914173,0.895408,0.877028,0.859026,0.841393
...,...,...,...,...,...,...,...,...,...,...,...
2544,2024-02-29,0.981576,0.963491,0.945740,0.928315,0.911936,0.895846,0.878743,0.861966,0.845510,0.829368
2545,2024-03-01,0.981528,0.963397,0.945600,0.928133,0.911712,0.895582,0.878464,0.861673,0.845203,0.829047
2546,2024-03-04,0.981682,0.963699,0.946046,0.928716,0.912401,0.896372,0.879368,0.862687,0.846322,0.830268
2547,2024-03-05,0.981682,0.963699,0.946046,0.928716,0.912401,0.896372,0.879402,0.862754,0.846421,0.830397


In [327]:
#DISCOUNT FACTORS 5 < T <=10 

df5 = pd.merge(df4, df3[['Dates', 'ACGB 10yr']], on='Dates', how='left') #add 10yr coupon column to DF data frame 

# Define the bond price equation with the previously calculated discount factors and set up to solve for for B(0,10) 
def bond_price_equation(x, coupon, df_05y, df_1y, df_15y, df_2y, df_25y, df_3y, df_35y, df_4y, df_45y, df_5y, FV=100, price=100):
    return (
        (coupon / 2) * df_05y + 
        (coupon / 2) * df_1y + 
        (coupon / 2) * df_15y + 
        (coupon / 2) * df_2y + 
        (coupon / 2) * df_25y + 
        (coupon / 2) * df_3y + 
        (coupon / 2) * df_35y + 
        (coupon / 2) * df_4y + 
        (coupon / 2) * df_45y + 
        (coupon / 2) * df_5y + 
        (coupon / 2) * (df_5y)**(9/10)*x**(1/10) + 
        (coupon / 2) * (df_5y)**(8/10)*x**(2/10) + 
        (coupon / 2) * (df_5y)**(7/10)*x**(3/10) + 
        (coupon / 2) * (df_5y)**(6/10)*x**(4/10) +
        (coupon / 2) * (df_5y*x)**(1/2) + 
        (coupon / 2) * (df_5y)**(4/10)*x**(6/10) + 
        (coupon / 2) * (df_5y)**(3/10)*x**(7/10) +
        (coupon / 2) * (df_5y)**(2/10)*x**(8/10) +
        (coupon / 2) * (df_5y)**(1/10)*x**(9/10) + 
        (FV + coupon / 2) * x - price
    )
# Define a function to calculate x

def calculate_x(coupon, df_05y, df_1y, df_15y, df_2y, df_25y, df_3y, df_35y, df_4y, df_45y, df_5y):
    from scipy.optimize import root_scalar
    result = root_scalar(bond_price_equation, args=(coupon, df_05y, df_1y, df_15y, df_2y, df_25y, df_3y, df_35y, df_4y, df_45y, df_5y), bracket=[0,1])
    return result.root
# Apply the function on the merged DataFrame
df5['10y DF'] = df5.apply(
    lambda row: calculate_x(row['ACGB 10yr'], row['0.5y DF'], row['1y DF'], row['1.5y DF'], row['2yr DF'], row['2.5y DF'], row['3y DF'], row['3.5y DF'], row['4y DF'], row['4.5y DF'], row['5y DF']),
    axis=1

)

#then work out B(0,5.5), B(0,6), B(0,6.5)...B(0,9.5) and store to discount factors df5
df5['5.5y DF'] = df5['5y DF']**(9/10)*df5['10y DF']**(1/10)
df5['6y DF'] = df5['5y DF']**(8/10)*df5['10y DF']**(2/10)
df5['6.5y DF'] = df5['5y DF']**(7/10)*df5['10y DF']**(3/10)
df5['7y DF'] = df5['5y DF']**(6/10)*df5['10y DF']**(4/10)
df5['7.5y DF'] = (df5['5y DF']*df5['10y DF'])**(1/2)
df5['8y DF'] = df5['5y DF']**(4/10)*df5['10y DF']**(6/10)
df5['8.5y DF'] = df5['5y DF']**(3/10)*df5['10y DF']**(7/10)
df5['9y DF'] = df5['5y DF']**(2/10)*df5['10y DF']**(8/10)
df5['9.5y DF'] = df5['5y DF']**(1/10)*df5['10y DF']**(9/10)

df5 = df5[['Dates','0.5y DF', '1y DF', '1.5y DF', '2yr DF', '2.5y DF', '3y DF', '3.5y DF', '4y DF', '4.5y DF', '5y DF', '5.5y DF', '6y DF', '6.5y DF', '7y DF', '7.5y DF', '8y DF', '8.5y DF', '9y DF', '9.5y DF', '10y DF']]
pd.set_option('display.max_columns', None)
df5 # This contains all of the discount factors for 0.5y,1y...9.5y, 10y (#Q1)

,Dates,0.5y DF,1y DF,1.5y DF,2yr DF,2.5y DF,3y DF,3.5y DF,4y DF,4.5y DF,5y DF,5.5y DF,6y DF,6.5y DF,7y DF,7.5y DF,8y DF,8.5y DF,9y DF,9.5y DF,10y DF
0,2013-09-02,0.987196,0.974556,0.962078,0.949760,0.934663,0.919807,0.901740,0.884028,0.866664,0.849641,0.829348,0.809539,0.790203,0.771329,0.752906,0.734923,0.717369,0.700235,0.683510,0.667184
1,2013-09-03,0.986914,0.973998,0.961252,0.948673,0.933340,0.918255,0.899998,0.882103,0.864565,0.847375,0.827028,0.807170,0.787789,0.768873,0.750411,0.732393,0.714807,0.697643,0.680892,0.664542
2,2013-09-04,0.986612,0.973403,0.960370,0.947513,0.931827,0.916402,0.897958,0.879886,0.862178,0.844826,0.824616,0.804890,0.785636,0.766843,0.748499,0.730594,0.713117,0.696058,0.679408,0.663155
3,2013-09-05,0.986427,0.973038,0.959831,0.946802,0.930883,0.915231,0.896665,0.878476,0.860656,0.843198,0.822754,0.802805,0.783341,0.764348,0.745816,0.727733,0.710088,0.692871,0.676072,0.659680
4,2013-09-06,0.986242,0.972673,0.959291,0.946093,0.929996,0.914173,0.895408,0.877028,0.859026,0.841393,0.820555,0.800233,0.780414,0.761086,0.742237,0.723854,0.705927,0.688444,0.671394,0.654766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2544,2024-02-29,0.981576,0.963491,0.945740,0.928315,0.911936,0.895846,0.878743,0.861966,0.845510,0.829368,0.810748,0.792545,0.774752,0.757357,0.740354,0.723732,0.707483,0.691599,0.676072,0.660893
2545,2024-03-01,0.981528,0.963397,0.945600,0.928133,0.911712,0.895582,0.878464,0.861673,0.845203,0.829047,0.810428,0.792228,0.774436,0.757044,0.740042,0.723422,0.707176,0.691294,0.675769,0.660592
2546,2024-03-04,0.981682,0.963699,0.946046,0.928716,0.912401,0.896372,0.879368,0.862687,0.846322,0.830268,0.811833,0.793807,0.776182,0.758948,0.742096,0.725619,0.709508,0.693754,0.678350,0.663289
2547,2024-03-05,0.981682,0.963699,0.946046,0.928716,0.912401,0.896372,0.879402,0.862754,0.846421,0.830397,0.812034,0.794078,0.776518,0.759346,0.742555,0.726134,0.710077,0.694375,0.679020,0.664004


In [489]:
#Q2) prices of the bonds. first set up the data frame 

import pandas as pd

Q2data = {
    "Bond": ['B1','B2','B3','B4'], 
    "Maturity":["21 April 2025", "21 April 2026", "21 May 2028", "21 December 2029"],
    "Maturity (years)": [2, 3, 61/12, 78/12],
    "Coupon": ['3.25%', '4.25%', '2.25%', '1.00%']
}
Q2datadf = pd.DataFrame(Q2data)

Q2datadf['Maturity'] = pd.to_datetime(Q2datadf['Maturity']) #change maturity type to datetime 
 
df5['Dates'] = pd.to_datetime(df5['Dates'])
specific_date = pd.to_datetime('2023-04-21') 
filtered_df = df5[df5['Dates'] == specific_date] # this shows the relevant discount factors for the 21 April 2023 date given in the question

# price of B1 and B2. We can use the previously computed discount factors for this date as the coupon schedule occurs every 6m which is the discount rates we have calculated

couponb1 = 3.25/2
couponb2 = 4.25/2

facevalue = 100

P1 = couponb1*filtered_df["0.5y DF"] + couponb1*filtered_df["1y DF"] + couponb1*filtered_df["1.5y DF"] + couponb1*filtered_df["2yr DF"] + facevalue*filtered_df["2yr DF"]
P1 = float(P1.iloc[0]) #store as float
P2 = couponb2*filtered_df["0.5y DF"] + couponb2*filtered_df["1y DF"] + couponb2*filtered_df["1.5y DF"] + couponb2*filtered_df["2yr DF"] + couponb2*filtered_df["2.5y DF"] + \
    couponb2*filtered_df["3y DF"] + facevalue*filtered_df["3y DF"]
P2 = float(P2.iloc[0])

#for B3, we need to find the relevant discount factors for 1m,7m, 1y 1m... 5y 1m to match the coupon schedule (tenor is 5y 1 month) 
# important to interpolate these discount factors from the DFs where we used the root solvers i.e. B(0,2), B(0,3), B(0,5) and B(0,10)

df_first_7y = filtered_df.iloc[:] #renaming set
df_first_7y.columns = ['Dates', '6m DF', '12m DF', '18m DF', '24m DF', '30m DF', '36m DF', '42m DF', 
                               '48m DF', '54m DF', '60m DF', '66m DF', '72m DF', '78m DF', '84m DF', "90m DF", "96m DF", "102m DF", "108m DF", "114m DF", "120m DF"] # change column name to month format

df_first_7y["1m DF"] = df_first_7y['24m DF']**(1/24)
df_first_7y["7m DF"] = df_first_7y['24m DF']**(7/24)
df_first_7y["13m DF"] = df_first_7y['24m DF']**(13/24)
df_first_7y["19m DF"] = df_first_7y['24m DF']**(19/24)

df_first_7y["25m DF"] = df_first_7y['24m DF']**(11/12)*df_first_7y['36m DF']**(1/12)
df_first_7y["31m DF"] = df_first_7y['24m DF']**(5/12)*df_first_7y['36m DF']**(7/12)

df_first_7y["37m DF"] = df_first_7y['36m DF']**(23/24)*df_first_7y['60m DF']**(1/24)
df_first_7y["43m DF"] = df_first_7y['36m DF']**(17/24)*df_first_7y['60m DF']**(7/24)
df_first_7y["49m DF"] = df_first_7y['36m DF']**(11/24)*df_first_7y['60m DF']**(13/24)
df_first_7y["55m DF"] = df_first_7y['36m DF']**(5/24)*df_first_7y['60m DF']**(19/24)

df_first_7y["61m DF"] = df_first_7y['60m DF']**(59/60)*df_first_7y['120m DF']**(1/60)

# now calculate price since all relevant discount factors found
couponb3 = 2.25/2

P3 = couponb3*df_first_7y["1m DF"] + couponb3*df_first_7y["7m DF"] + couponb3*df_first_7y["13m DF"] + couponb3*df_first_7y["19m DF"] + couponb3*df_first_7y["25m DF"] + \
    couponb3*df_first_7y["31m DF"] + couponb3*df_first_7y["37m DF"] + couponb3*df_first_7y["43m DF"] + couponb3*df_first_7y["49m DF"] + couponb3*df_first_7y["55m DF"] + \
    couponb3*df_first_7y["61m DF"] + facevalue*df_first_7y["61m DF"]
P3 = float(P3.iloc[0])

#for B4, we need to find the relevant discount factors for 2m, 8m,1y 2m,... 6y 8m to match the coupon schedule (tenor is 6y 8 months).
# like the above, important to interpolate these discount factors from the DFs where we used the root solvers i.e. B(0,2), B(0,3), B(0,5) and B(0,10)

df_first_7y["2m DF"] = df_first_7y['24m DF']**(2/24)
df_first_7y["8m DF"] = df_first_7y['24m DF']**(8/24)
df_first_7y["14m DF"] = df_first_7y['24m DF']**(14/24)
df_first_7y["20m DF"] = df_first_7y['24m DF']**(20/24)

df_first_7y["26m DF"] = df_first_7y['24m DF']**(10/12)*df_first_7y['36m DF']**(2/12)
df_first_7y["32m DF"] = df_first_7y['24m DF']**(4/12)*df_first_7y['36m DF']**(8/12)

df_first_7y["38m DF"] = df_first_7y['36m DF']**(22/24)*df_first_7y['60m DF']**(2/24)
df_first_7y["44m DF"] = df_first_7y['36m DF']**(16/24)*df_first_7y['60m DF']**(8/24)
df_first_7y["50m DF"] = df_first_7y['36m DF']**(10/24)*df_first_7y['60m DF']**(14/24)
df_first_7y["56m DF"] = df_first_7y['36m DF']**(4/24)*df_first_7y['60m DF']**(20/24)

df_first_7y["62m DF"] = df_first_7y['60m DF']**(58/60)*df_first_7y['120m DF']**(2/60)
df_first_7y["68m DF"] = df_first_7y['60m DF']**(52/60)*df_first_7y['120m DF']**(8/60)
df_first_7y["74m DF"] = df_first_7y['60m DF']**(46/60)*df_first_7y['120m DF']**(14/60)
df_first_7y["80m DF"] = df_first_7y['60m DF']**(40/60)*df_first_7y['120m DF']**(20/60)

# now, find price of B4
couponb4 = 1/2
P4 = couponb4*df_first_7y["2m DF"] + couponb4*df_first_7y["8m DF"] + couponb4*df_first_7y["14m DF"] + couponb4*df_first_7y["20m DF"] + couponb4*df_first_7y["26m DF"] + \
    couponb4*df_first_7y["32m DF"] + couponb4*df_first_7y["38m DF"] + couponb4*df_first_7y["44m DF"] + couponb4*df_first_7y["50m DF"] + couponb4*df_first_7y["56m DF"] + \
    couponb4*df_first_7y["62m DF"] + couponb4*df_first_7y["68m DF"] + couponb4*df_first_7y["74m DF"] + couponb4*df_first_7y["80m DF"] + facevalue*df_first_7y["80m DF"]

P4 = float(P4.iloc[0])

Q2datadf["Price as of 21 April 2023"] = [P1, P2, P3, P4] #store in the data frame 
Q2datadf["Duration"] = '' #load other columns in there for next part of the question
Q2datadf["Convexity"] = '' 

Q2datadf # prices now stored in table

,Bond,Maturity,Maturity (years),Coupon,Price as of 21 April 2023,Duration,Convexity
0,B1,2025-04-21,2.000000,3.25%,100.419964,,
1,B2,2026-04-21,3.000000,4.25%,103.203597,,
2,B3,2028-05-21,5.083333,2.25%,96.540200,,
3,B4,2029-12-21,6.500000,1.00%,86.483072,,


In [491]:
#Q2: duration and Convexity (Fisher Weil versions)

#duration is sum(t*cashflow*discount factor)/price of bond for t in years. Proceed with 1st Bond and work way to 4th

dB1 = (0.5*couponb1*filtered_df["0.5y DF"] + 1*couponb1*filtered_df["1y DF"] + 1.5*couponb1*filtered_df["1.5y DF"] + 2*couponb1*filtered_df["2yr DF"] + 2*facevalue*filtered_df["2yr DF"])/P1
dB2 = (0.5*couponb2*filtered_df["0.5y DF"] + 1*couponb2*filtered_df["1y DF"] + 1.5*couponb2*filtered_df["1.5y DF"] + 2*couponb2*filtered_df["2yr DF"] + \
       2.5*couponb2*filtered_df["2.5y DF"] + 3*couponb2*filtered_df["3y DF"] + 3*facevalue*filtered_df["3y DF"])/P2
dB3 = ((1/12)*couponb3*df_first_7y["1m DF"] + (7/12)*couponb3*df_first_7y["7m DF"] + (13/12)*couponb3*df_first_7y["13m DF"] + (19/12)*couponb3*df_first_7y["19m DF"] +\
        (25/12)*couponb3*df_first_7y["25m DF"] + (31/12)*couponb3*df_first_7y["31m DF"] + (37/12)*couponb3*df_first_7y["37m DF"] + (43/12)*couponb3*df_first_7y["43m DF"] +\
        (49/12)*couponb3*df_first_7y["49m DF"] + (55/12)*couponb3*df_first_7y["55m DF"] + (61/12)*couponb3*df_first_7y["61m DF"] + (61/12)*facevalue*df_first_7y["61m DF"])/P3

dB4 = ((2/12)*couponb4*df_first_7y["2m DF"] + (8/12)*couponb4*df_first_7y["8m DF"] + (14/12)*couponb4*df_first_7y["14m DF"] + (20/12)*couponb4*df_first_7y["20m DF"] +\
        (26/12)*couponb4*df_first_7y["26m DF"] + (32/12)*couponb4*df_first_7y["32m DF"] + (38/12)*couponb4*df_first_7y["38m DF"] + (44/12)*couponb4*df_first_7y["44m DF"] +\
        (50/12)*couponb4*df_first_7y["50m DF"] + (56/12)*couponb4*df_first_7y["56m DF"] + (62/12)*couponb4*df_first_7y["62m DF"] + (68/12)*couponb4*df_first_7y["68m DF"] +\
        (74/12)*couponb4*df_first_7y["74m DF"] + (80/12)*couponb4*df_first_7y["80m DF"] + (80/12)*facevalue*df_first_7y["80m DF"])/P4

dB1 = float(dB1.iloc[0]) #store as floats
dB2 = float(dB2.iloc[0])
dB3 = float(dB3.iloc[0])
dB4 = float(dB4.iloc[0])

Q2datadf["Duration"] = [dB1, dB2, dB3, dB4]

# convexity is sum(t^2*cashflow*discount factor)/price of bond for t in years 

cB1 = ((0.5)**2*couponb1*filtered_df["0.5y DF"] + (1)**2*couponb1*filtered_df["1y DF"] + (1.5)**2*couponb1*filtered_df["1.5y DF"] + (2)**2*couponb1*filtered_df["2yr DF"] + \
(2)**2*facevalue*filtered_df["2yr DF"])/P1
cB2 = ((0.5)**2*couponb2*filtered_df["0.5y DF"] + (1)**2*couponb2*filtered_df["1y DF"] + (1.5)**2*couponb2*filtered_df["1.5y DF"] + (2)**2*couponb2*filtered_df["2yr DF"] + \
       (2.5)**2*couponb2*filtered_df["2.5y DF"] + (3)**2*couponb2*filtered_df["3y DF"] + (3)**2*facevalue*filtered_df["3y DF"])/P2

cB3 = ((1/12)**2*couponb3*df_first_7y["1m DF"] + (7/12)**2*couponb3*df_first_7y["7m DF"] + (13/12)**2*couponb3*df_first_7y["13m DF"] + (19/12)**2*couponb3*df_first_7y["19m DF"] +\
        (25/12)**2*couponb3*df_first_7y["25m DF"] + (31/12)**2*couponb3*df_first_7y["31m DF"] + (37/12)**2*couponb3*df_first_7y["37m DF"] + (43/12)**2*couponb3*df_first_7y["43m DF"] +\
        (49/12)**2*couponb3*df_first_7y["49m DF"] + (55/12)**2*couponb3*df_first_7y["55m DF"] + (61/12)**2*couponb3*df_first_7y["61m DF"] + (61/12)**2*facevalue*df_first_7y["61m DF"])/P3

cB4 = ((2/12)**2*couponb4*df_first_7y["2m DF"] + (8/12)**2*couponb4*df_first_7y["8m DF"] + (14/12)**2*couponb4*df_first_7y["14m DF"] + (20/12)**2*couponb4*df_first_7y["20m DF"] +\
        (26/12)**2*couponb4*df_first_7y["26m DF"] + (32/12)**2*couponb4*df_first_7y["32m DF"] + (38/12)**2*couponb4*df_first_7y["38m DF"] + (44/12)**2*couponb4*df_first_7y["44m DF"] +\
        (50/12)**2*couponb4*df_first_7y["50m DF"] + (56/12)**2*couponb4*df_first_7y["56m DF"] + (62/12)**2*couponb4*df_first_7y["62m DF"] + (68/12)**2*couponb4*df_first_7y["68m DF"] +\
        (74/12)**2*couponb4*df_first_7y["74m DF"] + (80/12)**2*couponb4*df_first_7y["80m DF"] + (80/12)**2*facevalue*df_first_7y["80m DF"])/P4

cB1 = float(cB1.iloc[0]) #store as floats
cB2 = float(cB2.iloc[0])
cB3 = float(cB3.iloc[0])
cB4 = float(cB4.iloc[0])

Q2datadf["Convexity"] = [cB1, cB2, cB3, cB4]

Q2datadf

,Bond,Maturity,Maturity (years),Coupon,Price as of 21 April 2023,Duration,Convexity
0,B1,2025-04-21,2.000000,3.25%,100.419964,1.952653,3.866048
1,B2,2026-04-21,3.000000,4.25%,103.203597,2.850887,8.380449
2,B3,2028-05-21,5.083333,2.25%,96.540200,4.777847,23.817872
3,B4,2029-12-21,6.500000,1.00%,86.483072,6.420423,42.288660


In [493]:
# in written form 

print(f"Price of B1: ${P1:.4f}")
print(f"Price of B2: ${P2:.4f}")
print(f"Price of B3: ${P3:.4f}")
print(f"Price of B4: ${P4:.4f}")

print(f"Duration of B1: {dB1:.6f}")
print(f"Duration of B2: {dB2:.6f}")
print(f"Duration of B3: {dB3:.6f}")
print(f"Duration of B4: {dB4:.6f}")

print(f"Convexity of B1: {cB1:.6f}")
print(f"Convexity of B2: {cB2:.6f}")
print(f"Convexity of B3: {cB3:.6f}")
print(f"Convexity of B4: {cB4:.6f}")


Price of B1: $100.4200
Price of B2: $103.2036
Price of B3: $96.5402
Price of B4: $86.4831
Duration of B1: 1.952653
Duration of B2: 2.850887
Duration of B3: 4.777847
Duration of B4: 6.420423
Convexity of B1: 3.866048
Convexity of B2: 8.380449
Convexity of B3: 23.817872
Convexity of B4: 42.288660


In [495]:
#Q3: first work out present value of the liability. Given this is 21 April 2023, this is 4 years in the future, the present of the liability is 120m * B(0,4) 

pvL = 120000000*filtered_df["4y DF"]
pvL = float(pvL.iloc[0]) 

#set up data points
pvL # present value of liability
dL = 4 #duration of liability (can just consider as ZCB)
P1 # price of B1
P4 # price of B4 
dB1 # duration of B1 
dB4 # duration of B4 


import numpy as np 

# 2 equations here: present value of hedge = present value of liability i.e. x*p1 + y*p4 = pvL, and 
# weighted duration of the hedge = weighted duration of the liability i.e. x*p1*db1 + y*p4*db2 = pvL*dL (net fisher/weil duration is 0)
# where x represents the position in B1 and y the position in B2

# Matrix form of the system of equations:
# [P1, P4]           [x]   = [pvL]
# [P1*dB1, P4*dB4]   [y]   = [pvL*dL]

# Coefficients matrix (A) and constants vector (B)
A = np.array([[P1, P4],
              [P1 * dB1, P4 * dB4]])

B = np.array([pvL, pvL*dL])

# Solve the system of linear equations (Ax = B) where x is a vector 
positions = np.linalg.solve(A, B)

# produce the output (x,y) that satisfies BOTH of the above equations
x, y = positions
print(f"Units of B1 (x): {x:.2f}")
print(f"Units of B4 (y): {y:.2f}")

print(f"Position in B1 at current market price: ${x*P1:.2f}")
print(f"Position in B4 at current market price: ${y*P4:.2f}")

Units of B1 (x): 570968.12
Units of B4 (y): 560791.00
Position in B1 at current market price: $57336598.32
Position in B4 at current market price: $48498929.09
